Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [17]:
#for p in minha_MLP.parameters():
#    print(p)

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[-0.1539],
        [-0.1496],
        [-0.1496],
        ...,
        [-0.1534],
        [-0.1507],
        [-0.1504]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    #print(epoca, loss.data)

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

#for yt, yp in zip(y_true, y_pred):
    #print(yt, yp)

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

#for yt, yp in zip(y_true, y_pred):
#    print(yt, yp)

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1399.9460621858798


## Conclusão

